<a href="https://colab.research.google.com/github/Dhanyabahadur/project_notebooks/blob/main/Llama_2_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Mar 10 12:30:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 19.5 MB/s eta 0:00:00


In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-creden

In [5]:
from langchain.llms import HuggingFacePipeline

In [6]:
from transformers import AutoTokenizer
import transformers
import torch

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
model = "meta-llama/Llama-2-7b-chat-hf"

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [10]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [11]:
llm = HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

In [12]:
prompt = "What would be a good name for a company that makes colorful socks"

In [13]:
print(llm(prompt))

?

Answer: Here are some suggestions for a company name that makes colorful socks:

1. SoleMates: This name plays on the idea of socks being a companion for your feet, and the word "mates" implies a fun, casual vibe.
2. HueHues: This name is a play on the word "hue," which means a color, and it also has a fun, quirky sound to it.
3. Footloose & Fancy-Free: This name is a play on the phrase "footloose and fancy-free," which means feeling free and unencumbered. It also implies that the socks are fun and carefree.
4. SockItToMe: This name is a play on the phrase "sock it to me," which means to impress or surprise someone. It also has a fun, energetic sound to it.
5. RainbowToes: This name is a play on the idea of rainbows being colorful and vibrant, and it also implies that the socks will bring a bit of color and joy to the wearer's toes.
6. SockSational: This name is a play on the word "sensational," and it implies that the socks are something special and exciting.
7. ToeTastic: This nam

In [14]:
prompt = "I want to open a restaurant for indian food. Suggest me a fence name for this."

In [15]:
print(llm(prompt))


Sounds like you have a great idea for a restaurant! Here are some suggestions for a catchy and memorable name for your Indian food restaurant:

1. Spice Route - This name plays off the idea of a spice route, which was a major trade route in the past that brought spices from India and other parts of Asia to Europe. It's a clever and catchy name that will immediately convey to customers what kind of food they can expect at your restaurant.
2. Tandoor - A tandoor is a clay oven used in Indian cuisine to cook a variety of dishes, including naan bread, kebabs, and more. Naming your restaurant after this popular cooking vessel will give it a distinctly Indian flavor.
3. Curry Leaf - The curry leaf is a common ingredient in many Indian dishes, and it's a great name for a restaurant that specializes in Indian food. It's easy to remember and will likely be associated with the flavors and aromas of Indian cuisine.
4. Mumbai Masala - Mumbai is the largest city in India and is known for its vibra

In [16]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [17]:
prompt_template1 = PromptTemplate(input_variables=['cuisine'],
                                  template="I want to open a restaurant for {cuisine} food. Suggest a fency name for this.")

In [18]:
input_prompt=prompt_template1.format(cuisine="indian")

In [19]:
print(input_prompt)

I want to open a restaurant for indian food. Suggest a fency name for this.


In [20]:
prompt_template2=PromptTemplate(input_variables=["book_name"],
                                template="Provide me a concise summary of the book {book_name}")

In [21]:
input_prompt=prompt_template2.format(book_name="Alchemist")

In [22]:
print(input_prompt)

Provide me a concise summary of the book Alchemist


In [25]:
chain = LLMChain(llm=llm, prompt=prompt_template1, verbose=True)
response = chain.run("Indian")
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Indian food. Suggest a fency name for this.

> Finished chain.

Answer: Sure, I'd be happy to help you come up with a catchy name for your Indian restaurant! Here are a few suggestions:

1. Tandoor Nights: This name suggests a cozy and inviting atmosphere, perfect for a restaurant that serves delicious Indian cuisine.
2. Spice Route: This name takes inspiration from the historic spice trade routes that connected India to other parts of the world, and suggests a restaurant that offers a flavorful and diverse menu.
3. Mumbai Morsels: This name is perfect for a restaurant that serves up tasty, bite-sized dishes that are characteristic of the vibrant city of Mumbai.
4. Naan Stop: This name is a play on the popular Indian flatbread naan, and suggests a restaurant that serves up a variety of tasty naan dishes, as well as other Indian favorites.
5. Tikka Tales: This name takes inspiration from the popu